In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


In [2]:
df = pd.read_csv(r"E:\Tài liệu học\2024-2\Nhập môn Học máy và khai phá dữ liệu\Diabetes-Assessment-and-Prediction\Data\diabetes_dataset_processed.csv")
df.head(5)

,Insulin Levels,Age,BMI,Blood Pressure,Cholesterol Levels,Waist Circumference,Blood Glucose Levels,Weight Gain During Pregnancy,Pancreatic Health,Pulmonary Function,...,Physical Activity_Moderate,History of PCOS_No,History of PCOS_Yes,Previous Gestational Diabetes_No,Previous Gestational Diabetes_Yes,Steroid Use History_No,Steroid Use History_Yes,Family History_No,Family History_Yes,Target
0,40,44,38,124,201,50,168,18,36,76,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,Steroid-Induced Diabetes
1,13,1,17,73,121,24,178,8,26,60,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,Neonatal Diabetes Mellitus (NDM)
2,27,36,24,121,185,36,105,15,56,80,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,Prediabetic
3,8,7,16,100,151,29,121,12,49,89,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,Type 1 Diabetes
4,17,10,17,103,146,33,289,2,10,41,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,Wolfram Syndrome


In [3]:
le = LabelEncoder()
df['Target'] = le.fit_transform(df['Target'])
X = df.drop(columns=['Target'])
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])

param_grid = [
    {
        'classifier__kernel': ['linear'],
        'classifier__C': [0.1, 1, 10],
        'classifier__class_weight': [None, 'balanced']
    },
    {
        'classifier__kernel': ['rbf'],
        'classifier__C': [0.1, 1, 10],
        'classifier__gamma': ['scale', 'auto'],
        'classifier__class_weight': [None, 'balanced']
    }
]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# scoring='precision_macro' để không bỏ sót người bệnh
# scoring='recall_macro' để không muốn dự đoán sai người khỏe mạnh là bị bệnh
# scoring='f1_macro' để cân bằng giữa precision và recall
best_model = GridSearchCV(pipeline, param_grid, cv=cv, scoring='precision_macro', n_jobs=-1)

best_model.fit(X_train, y_train)    
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)

              precision    recall  f1-score   support

           0       0.83      0.75      0.79      1070
           1       0.69      0.71      0.70      1072
           2       0.81      0.82      0.82      1072
           3       0.77      0.76      0.76      1194
           4       0.99      1.00      1.00      1018
           5       0.78      0.79      0.79      1089
           6       0.64      0.57      0.60      1053
           7       0.62      0.57      0.59      1048
           8       0.79      0.86      0.83      1123
           9       0.73      0.72      0.72      1083
          10       0.62      0.74      0.68      1064
          11       0.84      0.83      0.83      1063
          12       0.83      0.84      0.84      1051

    accuracy                           0.76     14000
   macro avg       0.77      0.77      0.76     14000
weighted avg       0.77      0.76      0.76     14000



In [5]:
import joblib

model_path = r"E:\Tài liệu học\2024-2\Nhập môn Học máy và khai phá dữ liệu\Diabetes-Assessment-and-Prediction\Model\Built_model\svm_model.pkl"

joblib.dump(best_model, model_path)

['E:\\Tài liệu học\\2024-2\\Nhập môn Học máy và khai phá dữ liệu\\Diabetes-Assessment-and-Prediction\\Model\\Built_model\\svm_model.pkl']

In [8]:
disp.plot(cmap='Blues', values_format='d')
plt.xticks(rotation=60, ha='right')
plt.savefig(r'E:\Tài liệu học\2024-2\Nhập môn Học máy và khai phá dữ liệu\Diabetes-Assessment-and-Prediction\Result&Visualization\svm_heatmap.png', bbox_inches='tight')  
plt.close()